In [1]:
import simpy
from random import random
import networkx as nx
import sys

# Protocol:
class P(object):
    TYPE = 0
    HELLO_MSG = 0
    FORWARD_MSG = 1
    SWITCH_NAME = 1
    LSP = 1
    DV  = 1
    FROM_WHOM = 2
    ACK = 3
    COST = 2

def parseLS(ls):
    if ls is None: raise Exception('link state is none.')
    return (ls[0], ls[1], {'weight': ls[2]})
    
def makePrinter(log):
    def p(msgs):
        if log:
            content = ''
            for msg in msgs:
                content += ' ' + str(msg)
            print content
    return p

p = makePrinter(False)

# Define a Port
class Port(object):
    def __init__(self, env, delay=0):
        self.env = env
        self.store = simpy.Store(env)

    def send(self, msg):
        return self.store.put(msg)
    
    def receive(self):
        if self.size() == 0:
            return None 
        return self.store.get()
    
    def size(self):
        return len(self.store.items)

# Define a Link
class Link(object):
    def __init__(self, env, delay=0):
        self.env = env
        self.port1 = Port(env)
        self.port2 = Port(env)
        self.delay = delay
        self.hasConnnected = False
                           
    def connect(self, switch1, switch2):
        if not self.hasConnnected:
            portForSwitch1 = {'in': self.port1, 'out': self.port2, 'delay': self.delay}
            switch1.addPort(portForSwitch1)
            portForSwitch2 = {'in': self.port2, 'out': self.port1, 'delay': self.delay}
            switch2.addPort(portForSwitch2)
            self.hasConnnected = True
        else:
            raise Exception('Already connected to switches')

# Define a switch
class Switch(object):
    UNIT = 0.01
    def __init__(self, env, name, reporter): #measureF=None):
        self.name = name
        self.ports = []
        self.reporter = reporter
        self.env = env
        # self.buf = {}
        self.paths = None
        self.interval = 10
        self.counter  = self.interval
        self.dv = {}
    
    def countdown(self):
        self.counter -= 1
    
    def resetCounter(self):
        self.counter = self.interval
    
    def launch(self):
        self.env.process(self.run())
    
    def addPort(self, port):
        port['#'] = len(self.ports)
        self.ports.append(port)

    def _send(self, port, msg):
        yield port['out'].send((msg, self.env.now))
                           
    def _fecth(self, port):
        return port['in'].receive()

    def __str__(self):
        return self.name
    
    def run(self):
        thens = {}
        msgs  = {}
        
        for port in self.ports:
            msg = [P.HELLO_MSG, self.name, 0]
            # the cost can be redefined
            cost = port['delay']
            msg[P.COST] = cost
            p((port['#'], 'from', self.name))
            yield self.env.process(self._send(port, msg))
            
            thens[str(port)] = None
            msgs[str(port)] = None
        
        while True:
            yield self.env.timeout(Switch.UNIT)
            self.countdown()
            
            if len(self.ports) == 0: continue
            
            # periodically send distance vectors..
            if self.counter == 0:
                self.resetCounter()
                for port_ in self.ports:
                    ack_num = random()
                    msg = [P.FORWARD_MSG, self.dv.copy(), self.name, ack_num]
                    yield self.env.process(self._send(port_, msg))

            for port in self.ports:
#                 do_fetch = self._fecth(port)
#                 if do_fetch is None: continue
#                 # otherwise
#                 newMsg = yield do_fetch
                if thens[str(port)] is None:
                    do_fetch = self._fecth(port)
                    if do_fetch is None: continue
                    (msgs[str(port)], thens[str(port)]) = yield do_fetch
                
                if self.env.now - thens[str(port)] < port['delay']: continue
                else: thens[str(port)] = None

                newMsg = msgs[str(port)]

                if newMsg[P.TYPE] is P.HELLO_MSG:
                    # ADD THE SWITCH TO TOPOLOGY
                    switch = newMsg[P.SWITCH_NAME]
                    delay  = newMsg[P.COST]
                    self.dv[switch] = {'dist': delay, 'next_hop': switch}
                    p(('Received HELLO from:', switch))
                    
                elif newMsg[P.TYPE] is P.FORWARD_MSG:
                    source = newMsg[P.FROM_WHOM]
                    # ignore it.. otherwise there will be loops
                    if source == self.name: pass
                    else:
                        p((self.env.now, 'Received distance vectors from %s:' % (source,), newMsg))
                        dv = newMsg[P.DV]
                        
                        changed = False
                        for s in dv.keys():
                            if s not in self.dv.keys():
                                if s == self.name: continue
                                else:
                                    self.dv[s] = {'dist': self.dv[source]['dist'] + \
                                                  dv[s]['dist'], 'next_hop': source}
                                    changed = True
                            else: 
                                d = self.dv[s]['dist']
                                if d > self.dv[source]['dist'] + dv[s]['dist']:
                                    self.dv[s]['next_hop'] = source
                                    self.dv[s]['dist'] = self.dv[source]['dist'] + dv[s]['dist']
                                    changed = True
                        if changed: self.reporter[self.name] = self.env.now        
                    

class Map(object):
    def __init__(self):
        self.env = simpy.Environment()
        self.stations = {}
        self.lines = []
        self.convRecords = {}
        
    def load(self, statPairs):
        for i in xrange(len(statPairs)-1):
        	self.addStation(statPairs[i][0],  \
                            statPairs[i+1][0],\
                            statPairs[i][1])

    def addStation(self, stationA, stationB, delay):
        if (stationA, stationB) in self.lines or (stationB, stationA) in self.lines: return
        self.lines.append((stationA, stationB))
        
        if stationA not in self.stations:
            self.stations[stationA] = Switch(self.env, stationA, self.convRecords)
            self.convRecords[stationA] = 0
        stationA = self.stations[stationA]
        
        if stationB not in self.stations:
            self.stations[stationB] = Switch(self.env, stationB, self.convRecords)
            self.convRecords[stationB] = 0
        stationB = self.stations[stationB]

        l = Link(self.env, delay=delay)    
        l.connect(stationA, stationB)

    def launch(self, maxTime=10000):
        for station in self.stations.values():
            station.launch()
        self.env.run(until=maxTime)

def query_route(stations, source, sink):
    if source == sink: return []
    next_hop = source
    path = [source]
    while next_hop != sink:
        next_hop = stations[next_hop].dv[sink]['next_hop']
        path.append(next_hop)
    return (path, {'len': stations[source].dv[sink]['dist']})
        
    
# main():
import lines

stationMap = Map()
for line in lines.LINES:
    stationMap.load(line)
stationMap.launch(maxTime=200)

convergeTime = max(stationMap.convRecords.items(), key=lambda e: e[1])[1]

print 'When to converge:', convergeTime

When to converge: 105.4


In [2]:
print len(stationMap.stations['LingKongLu'].dv)
print len(stationMap.stations)

95
96


In [3]:
print 'Path from zjb to sht with least delay:', query_route(stationMap.stations, 'YiShanLu', 'JingAnSi')

Path from zjb to sht with least delay: (['YiShanLu', 'HongQiaoLu', 'YanAnXiLu', 'ZhongShanGongYuan', 'JiangSuLu', 'JingAnSi'], {'len': 12})
